In [2]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.filters.morningstar import Q1500US
from quantopian.pipeline.data.sentdex import sentiment
from quantopian.pipeline.data.morningstar  import operation_ratios


def make_pipeline():
    testing_factor=operation_ratios.revenue_growth.latest
    
    universe=(Q1500US()&testing_factor.notnull())
    
    testing_factor=testing_factor.rank(mask=universe,method='average')
    pipe=Pipeline(columns={"testing_factor":testing_factor},
                 screen=universe)
    
    return pipe
result=run_pipeline(make_pipeline(),start_date='2015-01-01',end_date='2016-01-01')
result.head()
    


testing_factor
2015-01-02 00:00:00+00:00 Equity(2 [ARNC])            755.0
                          Equity(24 [AAPL])           946.0
                          Equity(41 [ARCB])          1003.0
                          Equity(62 [ABT])            597.0
                          Equity(67 [ADSK])           912.0

In [3]:
assets=result.index.levels[1].unique()
pricing=get_pricing(assets,start_date='2014-12-01',end_date='2016-01-01',fields="open_price")
len(assets)

1734

In [ ]:
import alphalens

factor_data = alphalens.utils.get_clean_factor_and_forward_returns(factor = result['testing_factor'],
                                        prices = pricing["open_price"],
                                        quantiles = 2,
                                        periods = (3,10,30))

alphalens.tears.create_full_tear_sheet(factor_data)

Dropped 12.8% entries from factor data: 12.8% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
Quantiles Statistics


,min,max,mean,std,count,count %
factor_quantile,,,,,,
1,1.0,754.0,373.595137,215.433377,164521,50.014896
2,737.0,1500.0,1119.589553,215.189735,164423,49.985104


Returns Analysis


,3D,10D,30D
Ann. alpha,0.034,0.037,0.046
beta,0.001,-0.006,0.009
Mean Period Wise Return Top Quantile (bps),3.203,3.382,4.215
Mean Period Wise Return Bottom Quantile (bps),-3.199,-3.382,-4.229
Mean Period Wise Spread (bps),6.402,6.764,8.445
